In [1]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_validate
import numpy as np
import xgboost as xgb
import pandas as pd

In [2]:
train_datasetL = pd.read_csv("../data/ori_data/train_process.csv", header=None, sep="\t").iloc[:, 0].values
dev_datasetL = pd.read_csv("../data/ori_data/dev_process.csv", header=None, sep="\t").iloc[:, 0].values

In [3]:
train_datamatrix = np.load("../data/ori_data/train.featurematrix.data")
dev_datamatrix = np.load("../data/ori_data/dev.featurematrix.data")

In [21]:
from sklearn.ensemble import RandomForestClassifier

randomforest_classifier = RandomForestClassifier(n_estimators=10, criterion='gini', max_depth=None, 
                                                 min_samples_split=2, min_samples_leaf=1, 
                                                 min_weight_fraction_leaf=0.0, max_features='auto', 
                                                 max_leaf_nodes=None, min_impurity_decrease=0.0, 
                                                 min_impurity_split=None, bootstrap=True, oob_score=False, n_jobs=1, 
                                                 random_state=None, verbose=0, warm_start=False, class_weight="balanced")
randomforest_classifier.fit(train_datamatrix, train_datasetL)

RandomForestClassifier(bootstrap=True, class_weight='balanced',
            criterion='gini', max_depth=None, max_features='auto',
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [22]:
preds = randomforest_classifier.predict(dev_datamatrix)
pred_label = preds >= 0.5
pred_label = pred_label.astype(int)
from sklearn.metrics import classification_report
print(classification_report(dev_datasetL, pred_label))

             precision    recall  f1-score   support

          0       0.83      0.97      0.90     16751
          1       0.52      0.14      0.22      3744

avg / total       0.78      0.82      0.77     20495



In [23]:
from sklearn.linear_model import LogisticRegression
linear_classifier = LogisticRegression(class_weight="balanced", max_iter=1000)
linear_classifier.fit(train_datamatrix, train_datasetL)

LogisticRegression(C=1.0, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=1000,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

In [24]:
preds = linear_classifier.predict(dev_datamatrix)
pred_label = preds >= 0.5
pred_label = pred_label.astype(int)
from sklearn.metrics import classification_report
print(classification_report(dev_datasetL, pred_label))

             precision    recall  f1-score   support

          0       0.90      0.68      0.78     16751
          1       0.32      0.67      0.44      3744

avg / total       0.80      0.68      0.72     20495



In [25]:
import pickle
pickle.dump(linear_classifier, open("../data/m_result/linear_classifier.model", "wb"), 2)

In [43]:
# specify parameters via map
xgb_classifier = xgb.XGBClassifier(max_depth=11, learning_rate=0.01, n_estimators=1000, 
                                   silent=0, objective='binary:logistic', booster='gbtree', 
                                   n_jobs=1, nthread=None, gamma=0, min_child_weight=2, max_delta_step=0, subsample=1, 
                                   colsample_bytree=1, colsample_bylevel=1, reg_alpha=0, 
                                   reg_lambda=1, scale_pos_weight=4, 
                                   base_score=0.5, random_state=0, seed=None, missing=None)

xgb_classifier.fit(train_datamatrix, train_datasetL)

In [44]:
# make prediction
preds = xgb_classifier.predict(dev_datamatrix)

pred_label = preds >= 0.5
pred_label = pred_label.astype(int)
from sklearn.metrics import classification_report
print(classification_report(dev_datasetL, pred_label))

             precision    recall  f1-score   support

          0       0.88      0.84      0.86     16751
          1       0.40      0.48      0.44      3744

avg / total       0.79      0.78      0.78     20495



/home/xueyunzhe/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [42]:
pickle.dump(bst, open("../data/m_result/xgboost_3.model", "wb"), 2)

In [46]:
from sklearn.neighbors import KNeighborsClassifier

knn_classifier = KNeighborsClassifier(n_neighbors=5, weights='uniform', 
                                      algorithm='auto', leaf_size=30, 
                                      p=2, metric='minkowski', 
                                      metric_params=None, n_jobs=1)
knn_classifier.fit(train_datamatrix, train_datasetL)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')

In [47]:
# make prediction
preds = knn_classifier.predict(dev_datamatrix)

pred_label = preds >= 0.5
pred_label = pred_label.astype(int)
from sklearn.metrics import classification_report
print(classification_report(dev_datasetL, pred_label))

             precision    recall  f1-score   support

          0       0.83      0.94      0.89     16751
          1       0.40      0.16      0.23      3744

avg / total       0.75      0.80      0.77     20495



In [57]:
# select same number of pos and neg example

def select_trainset(datasetM, datasetL, neg_ratio=1):
    
    pos_index_list = []
    for index, val in enumerate(datasetL):
        if val == 1:
            pos_index_list.append(index)
    
    pos_trainM =  datasetM[pos_index_list, :]
    pos_trainL =  datasetL[pos_index_list]
    
    neg_index_list = [index for index in range(len(datasetL)) if index not in pos_index_list]
    
    
    neg_len = len(pos_index_list) * neg_ratio
    permuation_index = np.random.permutation(len(neg_index_list))
    neg_index_selected = permuation_index[:neg_len]
    
    neg_trainM = datasetM[neg_index_selected, :]
    neg_trainL = datasetL[neg_index_selected]
    
    return np.concatenate((pos_trainM, neg_trainM)), np.concatenate((pos_trainL, neg_trainL))

In [58]:
selected_trainM, selected_trainL = select_trainset(train_datamatrix, train_datasetL, neg_ratio=1)

print(selected_trainM.shape) 
print(selected_trainL.shape)

(29882, 88)
(29882,)


In [84]:
# specify parameters via map
xgb_classifier = xgb.XGBClassifier(max_depth=7, learning_rate=0.1, n_estimators=1000, 
                                   silent=0, objective='binary:logistic', booster='gbtree', 
                                   n_jobs=1, nthread=None, gamma=0, min_child_weight=2, max_delta_step=0, subsample=1, 
                                   colsample_bytree=1, colsample_bylevel=1, reg_alpha=0, 
                                   reg_lambda=1, scale_pos_weight=1, 
                                   base_score=0.5, random_state=0, seed=None, missing=None)

xgb_classifier.fit(selected_trainM, selected_trainL, 
                         eval_set=[(selected_trainM, selected_trainL), (dev_datamatrix, dev_datasetL)],
                         early_stopping_rounds=100, eval_metric="auc")

[0]	validation_0-auc:0.775401	validation_1-auc:0.718834
Multiple eval metrics have been passed: 'validation_1-auc' will be used for early stopping.

Will train until validation_1-auc hasn't improved in 100 rounds.
[1]	validation_0-auc:0.783165	validation_1-auc:0.727415
[2]	validation_0-auc:0.787075	validation_1-auc:0.728952
[3]	validation_0-auc:0.790219	validation_1-auc:0.731008
[4]	validation_0-auc:0.794188	validation_1-auc:0.733995
[5]	validation_0-auc:0.796786	validation_1-auc:0.735321
[6]	validation_0-auc:0.79857	validation_1-auc:0.735897
[7]	validation_0-auc:0.801264	validation_1-auc:0.737186
[8]	validation_0-auc:0.802788	validation_1-auc:0.737104
[9]	validation_0-auc:0.804593	validation_1-auc:0.73783
[10]	validation_0-auc:0.805867	validation_1-auc:0.738461
[11]	validation_0-auc:0.807701	validation_1-auc:0.739517
[12]	validation_0-auc:0.808946	validation_1-auc:0.740406
[13]	validation_0-auc:0.810856	validation_1-auc:0.740869
[14]	validation_0-auc:0.813216	validation_1-auc:0.741451

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=7, min_child_weight=2, missing=None, n_estimators=1000,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None, silent=0,
       subsample=1)

In [85]:
# make prediction
preds = xgb_classifier.predict(dev_datamatrix)

pred_label = preds >= 0.5
pred_label = pred_label.astype(int)
from sklearn.metrics import classification_report
print(classification_report(dev_datasetL, pred_label))

/home/xueyunzhe/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


             precision    recall  f1-score   support

          0       0.92      0.57      0.70     16751
          1       0.28      0.77      0.42      3744

avg / total       0.80      0.60      0.65     20495



In [86]:
pickle.dump(bst, open("../data/m_result/1_1_ratio_xgboost.model", "wb"), 2)